In [ ]:
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/ && echo 'Done'

In [ ]:
!kaggle datasets download -d jerrinbright/accident

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

In [ ]:
!unzip accident.zip

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
# YOUR CODE STARTS HERE    
    all_files = []
    for i in os.listdir(SOURCE):
        path = SOURCE + i
        if os.path.getsize(path):
            all_files.append(i)
    
    n = len(all_files)
    d = int(n * SPLIT_SIZE)
    rand = random.sample(all_files, n)
    train = rand[:d]
    valid = rand[d:]
    
    for i in train:
        copyfile(SOURCE + i, TRAINING + i)
        
    for i in valid:
        copyfile(SOURCE + i, TESTING + i)

In [ ]:
folders = [
    '/content/accident_detection',
    '/content/accident_detection/training',
    '/content/accident_detection/testing',
    '/content/accident_detection/training/accident',
    '/content/accident_detection/training/nonaccident',
    '/content/accident_detection/testing/accident',
    '/content/accident_detection/testing/nonaccident/',
]
for i in folders:
    try:
        os.mkdir(i)
    except OSError:
        pass

In [ ]:
accident_dir = '/content/ACCIDENT_DATASET/Accident/'
nonaccident_dir = '/content/ACCIDENT_DATASET/No Accident/'
train_accident_dir = '/content/accident_detection/training/accident/'
test_accident_dir = '/content/accident_detection/testing/accident/'
train_nonaccident_dir = '/content/accident_detection/training/nonaccident/'
test_nonaccident_dir = '/content/accident_detection/testing/nonaccident/'

split_size = .7
split_data(accident_dir, train_accident_dir, test_accident_dir, split_size)
split_data(nonaccident_dir, train_nonaccident_dir, test_nonaccident_dir, split_size)

In [ ]:
print('jumlah training img dengan kecelakaan : {}'.format(len(os.listdir(train_accident_dir))))
print('jumlah testing img dengan kecelakaan : {}'.format(len(os.listdir(test_accident_dir))))
print('jumlah training img tanpa kecelakaan : {}'.format(len(os.listdir(train_nonaccident_dir))))
print('jumlah testing img tanpa kecelakaan : {}'.format(len(os.listdir(test_nonaccident_dir))))

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])
path_model='model_kecelakaan.h5'
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
train_dir = '/content/accident_detection/training/'
train_datagen = ImageDataGenerator(rescale = 1.0/255.)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=10,
                                                    class_mode='binary',
                                                    target_size=(150, 150)) 

test_dir = '/content/accident_detection/testing/'
test_datagen = ImageDataGenerator(rescale = 1.0/255.)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  batch_size=10,
                                                  class_mode  = 'binary',
                                                  target_size = (150, 150))

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=15,
                              verbose=1,
                              validation_data=test_generator)

In [ ]:
%matplotlib inline
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
model.save("kecelakaan.h5")